In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd
import numpy as np
import csv
import ky_utils.ky_baseline_functions as base

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\ky-fl-combined\ky-model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/KY_cleaned").sort_values('person_id')
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-fl-data/FL_cleaned").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['violent_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['violent_six_month'].values

### Model

In [3]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [100,150,200]
depth = [7,8,9]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y, 
                     depth=depth, 
                     estimators=n_estimators,
                     seed=816)

#### XGBoost
learning_rate = [0.1]
depth = [4,5,6]
n_estimators = [100,150]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_violent6_ky_model = {"Logistic": logistic_summary,
                             "Lasso": lasso_summary,
                             "LinearSVM": svm_summary,
                             "RF": rf_summary,
                             "XGBoost": xgb_summary}

C:\Users\binha\Anaconda3\lib\site-packages\sklearn\calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
C:\Users\binha\Anaconda3\lib\site-packages\sklearn\calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
C:\Users\binha\Anaconda3\lib\site-packages\sklearn\calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
C:\Users\binha\Anaconda3\lib\site-packages\sklearn\calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
C:\Users\binha\Anaconda3\lib\site-packages\sklearn\calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
C:\Users\binha\Anaconda3\lib\site-packages\sklearn\calibration.py:455: RuntimeWarning: invalid value encountered in multiply
  TEP_minus_T1P = P * (T * E - T1)
C:\Users\binha\Anaconda3\lib\site-packag

In [4]:
results = []
for model_name, model_summary in summary_violent6_ky_model.items():
    results.append([model_name, 
                    np.mean(model_summary['KY_validation']), 
                    np.mean(model_summary['auc_diff']), 
                    np.mean(model_summary['FL_score'])])
results

[['Logistic', 0.8657332752798863, 0.0014507229062892036, 0.6337340989548925],
 ['Lasso', 0.865585177480303, 0.0012870551939129892, 0.6335301590527974],
 ['LinearSVM', 0.8638427909412882, 0.0013870286725506942, 0.6253535918153023],
 ['RF', 0.8772610784533665, 0.025684046049689303, 0.6549147145010756],
 ['XGBoost', 0.8803683906421436, 0.014471643066735007, 0.6262946965462337]]

In [5]:
path = "./results/baselines/six-month/"
results = [["Violent",             
            np.str(round(np.mean(logistic_summary['FL_score']),3)) + " (" + np.str(round(np.std(logistic_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(lasso_summary['FL_score']), 3)) + " (" + np.str(round(np.std(lasso_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(svm_summary['FL_score']),3)) + " (" + np.str(round(np.std(svm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(rf_summary['FL_score']),3)) + " (" + np.str(round(np.std(rf_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(xgb_summary['FL_score']),3)) + " (" + np.str(round(np.std(xgb_summary['FL_score']),3)) + ")"]]
with open(path + 'six-month-ky-baseline.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)